<center>
<img src="https://raw.githubusercontent.com/afrisenti-semeval/afrisent-semeval-2023/main/afrisenti-logo.png" width="30%" />
</center>

In [1]:
import argparse
import os

# parser = argparse.ArgumentParser()
# parser.add_argument("--lang_code",
#                         default='am',
#                         type=str,
#                         help="Valid codes: 'am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo'")

# args = parser.parse_args()

# LANGUAGE_CODE = args.lang_code

LANGUAGE_CODE = 'am'

usage: ipykernel_launcher.py [-h] [--lang_code LANG_CODE]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/thomas/Library/Jupyter/runtime/kernel-eb2d18ae-8c5c-42d6-89d2-48e889a29f62.json


SystemExit: 2

/Users/thomas/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
print("Language Code: ", LANGUAGE_CODE)

Language Code:  am


In [6]:
import pandas as pd
import numpy as np

# Please don not edit anything here
languages = ['am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo']

colab = False


TASK = 'SubtaskA'

In [7]:
# import os

# if colab:
#     from google.colab import drive
#     drive.mount('/content/drive')
#     proj_folder = '/content/drive/MyDrive'
# else:
#     proj_folder = os.getcwd()

# %cd {proj_folder}


# PROJECT_DIR = f'{proj_folder}/afrisent-semeval-2023'
# if not os.path.isdir(PROJECT_DIR):
#   %run Make_Datasets.py

In [8]:
# MODEL_NAME_OR_PATH = 'Davlan/afro-xlmr-mini'
MODEL_NAME_OR_PATH = 'xlm-roberta-base'
BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 8
LEARNING_RATE = 5e-5
NUMBER_OF_TRAINING_EPOCHS = 5
MAXIMUM_SEQUENCE_LENGTH = 128
SAVE_STEPS = -1



####Starter Code: Datasets, etc


In [9]:
from types import SimpleNamespace
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import warnings
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, balanced_accuracy_score
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim

import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
import numpy as np
import pandas
import pandas as pd
from datasets import load_dataset

import evaluate
import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    PreTrainedTokenizerFast,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from tokenizers import SentencePieceBPETokenizer
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version
from datasets import Features, Value, ClassLabel, load_dataset, Dataset

require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/text-classification/requirements.txt")

np.random.seed(69)
torch.manual_seed(420);

In [10]:
# folder = ''


# if colab:
#     from google.colab import drive
#     drive.mount('/content/drive')
#     proj_folder = '/content/drive/MyDrive'
# else:
#     proj_folder = os.getcwd()

# %cd {proj_folder}


# PROJECT_DIR = f'{proj_folder}/afrisent-semeval-2023'


PROJECT_DIR = 'afrisent-semeval-2023'

TRAINING_DATA_DIR = os.path.join(PROJECT_DIR, TASK, 'train')
FORMATTED_TRAIN_DATA = os.path.join(TRAINING_DATA_DIR, 'formatted-train-data')

TRAINING_DATA_DIR

'afrisent-semeval-2023/SubtaskA/train'

In [11]:
MAXIMUM_SEQUENCE_LENGTH = 500
DATA_DIR = os.path.join(TRAINING_DATA_DIR, 'splitted-train-dev-test', LANGUAGE_CODE)

In [7]:
LANGUAGE_CODE

'am'

In [8]:
# Set seed before initializing model.
set_seed(42069)

# obtain train data
df = pd.read_csv(DATA_DIR + '/train.tsv', sep='\t')
df = df.dropna()
train_dataset = Dataset.from_pandas(df)
label_list = df['label'].unique().tolist()

# obtain dev data
df = pd.concat([pd.read_csv(DATA_DIR + '/dev.tsv', sep='\t'), pd.read_csv(DATA_DIR + '/test.tsv', sep='\t')])
df = df.dropna()
eval_dataset = Dataset.from_pandas(df)
label_list = df['label'].unique().tolist()

# Labels
num_labels = len(label_list)
print(label_list)

['neutral', 'positive', 'negative']


####Tokenization

In [9]:
config = AutoConfig.from_pretrained(
    MODEL_NAME_OR_PATH,
    num_labels=num_labels,
    cache_dir=None,
    revision='main',
    use_auth_token=None,
)
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME_OR_PATH,
    do_lower_case=None,
    cache_dir=None,
    use_fast=True,
    revision='main',
    use_auth_token=None,
)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME_OR_PATH,
    from_tf=False,
    config=config,
    cache_dir=None,
    revision="main",
    use_auth_token=None,
    ignore_mismatched_sizes=False,
)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

In [10]:
# Preprocessing the datasets
# Padding strategy
padding = "max_length"


label_to_id = None
label_to_id = {v: i for i, v in enumerate(label_list)}



In [11]:
def preprocess_function(examples):
    texts =(examples['text'],)
    result = tokenizer(*texts, padding=padding, max_length=MAXIMUM_SEQUENCE_LENGTH)
    
    if label_to_id is not None and "label" in examples:
        result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
    
    result['length'], result["tokenized"] = [], []
    for input_ids in result['input_ids']:
        toks = tokenizer.convert_ids_to_tokens(input_ids, skip_special_tokens=True)
        result['length'].append(len(toks)+2)
        result['tokenized'].append(' '.join(toks))
    return result

train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    desc="Running tokenizer on train dataset",
)

eval_dataset = eval_dataset.map(
    preprocess_function,
    batched=True,
    desc="Running tokenizer on validation dataset",
)

Running tokenizer on train dataset:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on validation dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
train_dataset, eval_dataset

(Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask', 'length', 'tokenized'],
     num_rows: 4188
 }),
 Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask', 'length', 'tokenized'],
     num_rows: 599
 }))

In [13]:
train_text, train_labels = train_dataset['tokenized'], train_dataset['label']

In [14]:
eval_text, eval_labels = eval_dataset['tokenized'], eval_dataset['label']

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [16]:
# Get the metric function
metric = evaluate.load("accuracy")

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)


data_collator = default_data_collator

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args = TrainingArguments(
        do_train=True,
        do_eval=True,
        per_device_train_batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        num_train_epochs=NUMBER_OF_TRAINING_EPOCHS,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        output_dir='tmp_trainer',
        save_steps=SAVE_STEPS,
        overwrite_output_dir=True
    ),
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Training

    
train_result = trainer.train(resume_from_checkpoint=None)
metrics = train_result.metrics
metrics["train_samples"] = len(train_dataset) 

# Evaluation

metrics = trainer.evaluate(eval_dataset=eval_dataset)

metrics["eval_samples"] = len(eval_dataset)

for key,value in metrics.items():
    print(key, ':', value)

splitted_A = os.path.join(PROJECT_DIR, 'SubtaskA', 'train', 'splitted-train-dev-test')

try:
    LANGUAGE_CODE
except NameError:
    LANGUAGE_CODE = 'combined'
else:
    pass

data = []
for lang in languages:
    eval_path = os.path.join(splitted_A, lang)
    df = pd.read_csv(eval_path + '/dev.tsv', sep='\t')
    df = df.dropna()
    lang_eval = Dataset.from_pandas(df)
    lang_eval = lang_eval.map(
        preprocess_function,
        batched=True,
        load_from_cache_file=True,
        desc="Running tokenizer on validation dataset",
    )

    predictions, labels, metrics = trainer.predict(lang_eval, metric_key_prefix="eval")

    if LANGUAGE_CODE == lang:
        f1 = (f1_score(labels, np.argmax(predictions, axis=1), average='weighted'))
        bal_acc = balanced_accuracy_score( labels, np.argmax(predictions, axis=1))

    data.append([LANGUAGE_CODE, lang, str(list(predictions)), str(list(labels))])
df = pd.DataFrame(data, columns=['source', 'target', 'predictions', 'labels'])
df.to_csv(f'{LANGUAGE_CODE}_preds.csv', index=False)


trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)
print(f"f1 score: {f1:.3}, balanced acc: {bal_acc:.3}")

The following columns in the training set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: length, text, tokenized. If length, text, tokenized are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\Thomas\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4188
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 8
  Total optimization steps = 650


Step,Training Loss
500,1.042900




Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: length, text, tokenized. If length, text, tokenized are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 599
  Batch size = 8


Running tokenizer on validation dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: length, text, tokenized. If length, text, tokenized are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 599
  Batch size = 8


...

Running tokenizer on validation dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: length, text, tokenized. If length, text, tokenized are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 181
  Batch size = 8


Running tokenizer on validation dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: length, text, tokenized. If length, text, tokenized are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 852
  Batch size = 8


***** eval metrics *****
  eval_accuracy           =     0.3685
  eval_loss               =     1.1609
  eval_runtime            = 0:00:14.85
  eval_samples_per_second =     57.363
  eval_steps_per_second   =      7.204


In [17]:
# import os
# %cd {PROJECT_DIR}

C:\Users\Thomas\Downloads\afrisent\afrisenti-10701\baselines\afrisent-semeval-2023


In [18]:
# DATA_DIR = os.path.join(TRAINING_DATA_DIR, 'splitted-train-dev-test', LANGUAGE_CODE)
# OUTPUT_DIR = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE + '_no_test')
# kinya = 'jean-paul/KinyaBERT-small'

# !python starter_kit/run_textclass.py \
#   --model_name_or_path {kinya} \
#   --data_dir {DATA_DIR} \
#   --do_train \
#   --do_eval \
#   --per_device_train_batch_size {BATCH_SIZE} \
#   --learning_rate {MAXIMUM_SEQUENCE_LENGTH} \
#   --num_train_epochs {NUMBER_OF_TRAINING_EPOCHS} \
#   --max_seq_length {MAXIMUM_SEQUENCE_LENGTH} \
#   --output_dir {'tmp_trainer'} \
#   --save_steps {SAVE_STEPS} \
#   --overwrite_output_dir

11/26/2022 20:07:20 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
11/26/2022 20:07:20 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_back

NOTE: Redirects are currently not supported in Windows or MacOs.
[INFO|configuration_utils.py:681] 2022-11-26 20:07:20,993 >> loading configuration file https://huggingface.co/jean-paul/KinyaBERT-small/resolve/main/config.json from cache at C:\Users\Thomas/.cache\huggingface\transformers\a719f6bda3244524290766976d8ff648934ab590dd28e7f59e1f975787c3fa11.4378bcc314f6423d41284fcf6de531169f6a007f69606823cecc5ea3eac640ee
[INFO|configuration_utils.py:730] 2022-11-26 20:07:20,997 >> Model config BertConfig {
  "_name_or_path": "jean-paul/KinyaBERT-small",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "lay


  train_samples_per_second =     33.252
  train_steps_per_second   =      8.313
11/26/2022 20:17:54 - INFO - __main__ - *** Evaluate ***
***** eval metrics *****
  epoch                   =        5.0
  eval_accuracy           =     0.5142
  eval_loss               =  8510.1035
  eval_runtime            = 0:00:05.42
  eval_samples            =        599
  eval_samples_per_second =    110.496
  eval_steps_per_second   =     13.835



 10%|9         | 500/5235 [01:01<09:26,  8.35it/s]
                                                  

 15%|#5        | 801/5235 [01:37<08:48,  8.39it/s]


 19%|#9        | 1000/5235 [02:01<08:24,  8.39it/s]
                                                   

 29%|##8       | 1500/5235 [03:01<07:28,  8.33it/s]
                                                   

 38%|###8      | 2000/5235 [04:01<06:25,  8.39it/s]
                                                   

 48%|####7     | 2500/5235 [05:01<05:28,  8.33it/s]
                                                   

 57%|#####7    | 3000/5235 [06:01<04:26,  8.38it/s]
                                                   

 67%|######6   | 3500/5235 [07:01<03:28,  8.33it/s]
                                                   

 76%|#######6  | 4000/5235 [08:01<02:28,  8.34it/s]
                                                   

 86%|########5 | 4500/5235 [09:01<01:28,  8.35it/s]
                                                   

 91%|#########1| 4777/5235 [09:34<00:54,  8.35it/s]


 96%|#########5| 5000/5235 [10:01<00:28,  8.34it/s]
                                                   

100%|##########| 5235/5235 [10:29<00:00,  8.34it/s][INFO|trainer.py:1850] 2022-11-26 20:17:54,151 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)



                                                   

100%|##########| 5235/5235 [10:29<00:00,  8.31it/s]
[INFO|trainer.py:2640] 2022-11-26 20:17:54,163 >> Saving model checkpoint to tmp_trainer
[INFO|configuration_utils.py:451] 2022-11-26 20:17:54,163 >> Configuration saved in tmp_trainer\config.json
[INFO|modeling_utils.py:1566] 2022-11-26 20:17:54,540 >> Model weights saved in tmp_trainer\pytorch_model.bin
[INFO|tokenization_utils_base.py:2145] 2022-11-26 20:17:54,540 >> tokenizer config file saved in tmp_trainer\tokenizer_config.json
[INFO|tokenization_utils_base.py:2152] 2022-11-26 20:17:54,540 >> Special tokens file saved in tmp_trainer\special_tokens_map.json
[INFO|trainer.py:722] 2022-11-26 

100%|##########| 75/75 [00:05<00:00, 14.14it/s]
